# The impact of universal basic income on likely voters


## Nate Golden   |   June 2021

# Abstract

While previous UBI Center research has focused on the policy implications of a UBI it is important to also discuss its politics. Most non-UBI proposals from politicians involve an array of complicated tax credits and deductions that leave many voters confused about how they are impacted personally by the policy. A UBI funded by simple taxation would not only bring more winners than losers, it would be clear to the winners that they will benefit from the plan. In this paper I simulate two different UBI policies and find that about two-thirds of Americans would be better off with a budget neutral UBI funded by a flat tax on income. However, not all Americans are eligible so I zoom in on eligible and predicted voters and find that 59 percent and 53 percent would be winners respectively. Those numbers rise to 65 and 60 percent if non-voting populations (children and non-citizens) are excluded from a UBI. Political research suggests that there is some link between a person’s economic self-interest and their vote suggesting that there may be political benefits to targeting eligible voters and demonstrating the natural tension between policy and politics.


# Introduction

Unlike more complicated policies, a flat income tax funded UBI creates clear cut-off points for winners and losers based solely on a family's size and income. Empirical evidence has shown that voters are at least partially motivated to vote in their own economic self-interest. Thus, there may be some political advantages to UBI proposals that create a majority of winners. Yet, the percentage of winners from UBI proposals does not not perfectly align with support. A recent [review](https://www.jainfamilyinstitute.org/assets/how-to-frame-guaranteed-income-policy-lit-review-jfi.pdf) of UBI polling conducted by the Jain Family Institute found that support for a UBI typically ranges between 40 and 50 percent. Literature suggests that these numbers could rise if voters trust that the UBI will materially benefit their lives.

These findings may have implications for the 2022 congressional elections. If the Democrats maintain or build on their majority, they may be able to pass a larger share of President Biden’s agenda. If Republicans are able to grab control of either chamber, they will be able to block the Democrats plans until the next presidential election. With so much on the line, party strategists will be developing plans to win voters, particularly in key battleground states. One strategy is simply to run on policy reforms that will benefit the voters and hope that this will sway their decision. In the following section of this paper I conduct a literature review on whether voters act in their own self-interests.

# Are voters self-interested?

In 1984, Stanley Feldman, a professor at Stonybrook University, [analyzed](https://www.jstor.org/stable/586090) the existing research on whether people make political changes based on their own financial well-being. Feldman found competing evidence: some research suggested that voters did change behavior and others did not. Feldman ultimately concluded that voter decisions are modestly based on their own economic considerations, saying, “when government policies have a direct impact on them and they attribute responsibility to the government, people do alter their evaluations accordingly.” 

A decade later, Leonard Shabman and Kurt Stephenson questioned the validity of previous papers exploring self-interested voter theorems. The authors contended that preceding papers that used Census tract and voter precinct data do not accurately capture the behaviors of individuals. In their [paper](https://www.jstor.org/stable/4226891?seq=10#metadata_info_tab_contents), Shabman and Stephenson examined the impact of a bond proposal in Roanoke that would benefit about 10,000 out of the 100,000 residents. The taxes from the bill were estimated to increase the average resident's monthly utility bill by approximately $25, suggesting that the policy would have many more losers than winners. The bond passed with 56 percent support, though net winners from the bill were more likely to support the bill than net losers. The authors concluded that many voters are motivated by self-interest, but others do act more altruistically.

More recently, a 2021 [paper](http://economics.mit.edu/files/19248) from Antoine Levy at the Massachusetts Institute of Technology found that Emmanuel Macron’s promise to abolish a broad-based housing tax shifted voters most substantially in areas with higher median home prices. Levy used high frequency online search, polling, and prediction market data to show that the timing of the proposal corresponded with an increase in his polling numbers and his market-based predicted chances of victory.

In aggregate, empirical research suggests that the electorate does change their voting behavior in accordance to their own financial well-being and that proper messaging around proposals can further sway voters.


# Data and methodology

In the first part of my [analysis](https://colab.research.google.com/drive/1kXPES5SdYjIli9vCPOs-b3gnYGBsVCwc?usp=sharing) I ran data from the Current Population Survey voter supplements of November 2018 and 2020 through a Random Forest Classifier to create a model that predicted an adult citizens probability of voting on thirteen key demographics. A more detailed description of each demographic can be found at the Census Bureau's [website](https://www.census.gov/programs-surveys/cps.html). I then combined the vote predicting algorithm with the three years of data from the Census Bureau’s Annual Social and Economic Supplement (ASEC) to assign each individual a predicted probability of voting score ranging from 0 to 1. I used the economic data from the ASEC to simulate the impact of two different UBI models. The first model issued a flat tax of 10 percent and evenly redistributed the revenue to all Americans including children and non-citizens. The second model kept the same tax but only shared the dividends with adult citizens, as to target eligible voters.

Finally, I calculated the share of winners from each proposal across all 50 states and DC and divided my analysis into three categories: the full population, eligible voters (adult citizens), and predicted voters.

# Results

Figure 1 below displays the thirteen demographics used in my analysis and their feature importance. Age was the strongest indicator on probability to vote and sex was the weakest. 


In [4]:
# Import libraries
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import numpy as np
import microdf as mdf
import plotly.express as px
from ubicenter import format_fig

# Add Colors
BLUE = '#1976D2'
DARK_BLUE = '#1565C0'
LIGHT_BLUE = '#90CAF9'
GRAY = '#BDBDBD'

# Import data
# person = ASEC 2018, 2019, 2020
# voter = voter supplement 2018, 2020
voter = pd.read_csv('data/cps_asec_2017_2018_2019.csv.gz')
person = pd.read_csv('data/cps_voter_supplement_2018_2020.csv.gz')

# Lower columns
person.columns = person.columns.str.lower()
voter.columns = voter.columns.str.lower()

# Create Boolean voted column
voter["vote"] = voter.voted == 2

# Creating training dataframe as copy of voter supplement
train = voter.sample(frac=1.0).copy()
train["vote"] = train.voted == 2

# Train the data on the 11 key demographics
XCOLS = ['county','statefip', 'age', 'sex', 'race', 'marst', 'citizen', 'hispan', 'empstat', 'educ', 'faminc', 'hourwage', 'cbsasz' ]
rf = RandomForestClassifier().fit(train[XCOLS], train.vote, train.vosuppwt)

# Apply training results to ASEC to create a predicted vote score
preds = rf.predict_proba(person[XCOLS])
person["pred_vote"] = preds[:, 1]

# Set under 18 to 0, already 0 for non-citizen
person.loc[person.age < 18, "pred_vote"] = 0

# Display most relevant demographics
features = pd.Series(rf.feature_importances_, index=XCOLS).sort_values(ascending=True)
features = features.to_frame()
features = features.reset_index()
features.columns = ['category','importance']
features = features.round(3)

# Rename features for display
features = features.replace({'age': 'Age', 'educ': 'Education',
                            'statefip': 'State', 'faminc': 'Family income',
                            'county': 'County', 'citizen': 'Citizenship status',
                            'marst': 'Marriage status', 'empstat': 'Employment status',
                            'cbsasz': 'Core-based statistical area size', 
                            'race': 'Race', 'hispan': 'Hispanic', 
                            'hourwage': 'Hourly wage', 'sex': 'Sex'})

# Create features bar chart
fig = px.bar(features, x='importance', y = 'category', 
             title='Importance of features in a random forests model predicting voter turnout in 2018 and 2020',
             text='importance',
             orientation = 'h')

fig.update_layout(showlegend=False,
                  xaxis_ticksuffix='',
                  font=dict(family='Roboto'),
                  plot_bgcolor='white',
                 )

fig.update_xaxes(
        tickangle = 0,
        title_text = "Feature importance",
        tickfont = {"size": 14},
        title_standoff = 25,
        range=[0, 0.28]),

fig.update_yaxes(
        title_text = "",
        tickfont = {'size':14},
        title_standoff = 25)

fig.update_traces(texttemplate='%{text}', marker_color=BLUE)
format_fig(fig)

NameError: name 'format_fig' is not defined

I found that a flat tax funded UBI would benefit 65 percent of Americans but only 59 percent and 53 percent of eligible and predicted voters respectively. Because children and non-citizens cannot vote, including them in UBI proposals reduces the impact on the voting population. On the other hand, 60 percent of all Americans would be winners from a flat tax funded UBI that excluded children and non-citizens. However, these exclusions would generate higher returns for voters. 65 percent of eligible voters would be better off under this model and 60 percent of predicted voters. These differences detail how the optimal policy and optimal politics often do not align. The results for both simulations of every state can be found in Figure 5 of the appendix.


In [ ]:
# ASEC adjustments
person = person.rename(columns={'asecwt':'weight','statefip': 'state'})
person.adjginc.replace({99999999: 0},inplace=True)
person.weight /= 3
person.spmwt /= 3
population = person.weight.sum()

# Define age category
person['child'] = person.age < 18
person['adult'] = person.age >=18
person['person'] = 1

# Change fip codes to state codes for mapping
person['state'] = person['state'].astype(str)
person['state'].replace({'1':'AL','2':'AK', '4': 'AZ','5':'AR',
                         '6': 'CA', '8': 'CO', '9': 'CT',
                         '10':'DE', '11': 'DC', '12':'FL',
                         '13': 'GA','15':'HI', '16':'ID','17':'IL',
                         '18':'IN', '19':'IA','20':'KS', '21': 'KY',
                         '22':'LA', '23': 'ME', '24': 'MD',
                         '25':'MA', '26':'MI', '27': 'MN',
                         '28':'MS','29':'MO', '30': 'MT',
                         '31': 'NE', '32':'NV', '33': 'NH',
                         '34': 'NJ', '35': 'NM', '36':'NY',
                         '37':'NC', '38':'ND', '39': 'OH',
                         '40':'OK', '41': 'OR', '42':'PA',
                         '44':'RI','45':'SC', '46':'SD',
                         '47': 'TN', '48':'TX','49':'UT','50':'VT',
                         '51':'VA', '53':'WA', '54':'WV',
                         '55':'WI', '56':'WY'},inplace=True)

# Create voting eligible boolean column
person['eligible_voter'] = (person.age > 17) & (person.citizen < 5)

# Calculate total childrem=n, adults, and AGI for each SPM unit
group = person.groupby(['spmfamunit','year'])[['child','adult', 'adjginc', 'eligible_voter']].sum()
group.columns = ['total_children', 'total_adults', 'total_family_income', 'total_eligible_voter']
person = person.merge(group,left_on=['spmfamunit', 'year'], right_index=True)
person['total_people'] = person.total_adults + person.total_children

# Create dataframe with aggregated spm unit data
PERSON_COLUMNS = ['year', 'age', 'adjginc', 'person']
SPMU_COLUMNS = ['spmwt', 'spmtotres', 'spmfamunit', 'spmthresh', 'state']

spmu = person.groupby(SPMU_COLUMNS)[PERSON_COLUMNS].sum().reset_index()
spmu.rename(columns={'person':'numper'}, inplace=True)

# Calculate total AGI
total_agi = (spmu.adjginc * spmu.spmwt).sum()

# Create a 10% flat tax and revenue neutral UBI
# Note the amount of the flat tax does not matter for percent winners for a static model

flat_tax = 0.1
revenue = flat_tax * total_agi
ubi = revenue / population

# Determine if each individual is a winner from the flat tax and UBI
person['new_tax'] = person.total_family_income * flat_tax
person['total_ubi'] = ubi * person.total_people
person['change'] = person.total_ubi - person.new_tax
person['winner'] = person.change > 0

# Calculate the amount of eligible and predicted voters
eligible_population = (person.weight * person.eligible_voter).sum()
predicted_population = (person.weight * person.pred_vote).sum()

# Determine if each individual is a winner from only giving UBI to eligible voters
ubi_ev = revenue / eligible_population
person['total_ev_ubi'] = ubi_ev * person.total_eligible_voter
person['change_ev'] = person.total_ev_ubi - person.new_tax
person['winner_ev'] = person.change_ev > 0

# Create function with three outputs for each state
# 1) Overall percent winners
# 2) Eligible voters percent winners
# 3) Predicted voters percent winners

def winners_state(state):

  if state == 'US':
    target_persons = person.copy(deep=True)
  else:
    target_persons = person[person.state==state].copy(deep=True)

  target_population = target_persons.weight.sum()
  target_eligible_population = (target_persons.weight * target_persons.eligible_voter).sum()
  target_predicted_population = (target_persons.weight * target_persons.pred_vote).sum()

  total_winners = (target_persons.winner * target_persons.weight).sum()
  percent_total_winners = ((total_winners / target_population) * 100).round(1)

  total_eligible_winners = (target_persons.winner * target_persons.eligible_voter * target_persons.weight).sum()
  percent_eligible_winners = ((total_eligible_winners / target_eligible_population) * 100).round(1)

  total_predicted_winners = ((target_persons.winner * target_persons.weight *
                              target_persons.pred_vote).sum())
  
  percent_predicted_voters = ((total_predicted_winners / target_predicted_population) * 100).round(1)

  # Calculate share of winners for UBI only given to adult citizens

  total_winners_ev = (target_persons.winner_ev * target_persons.weight).sum()
  percent_total_winners_ev = ((total_winners_ev / target_population) * 100).round(1)

  total_eligible_winners_ev = (target_persons.winner_ev * target_persons.eligible_voter * target_persons.weight).sum()
  percent_eligible_winners_ev = ((total_eligible_winners_ev / target_eligible_population) * 100).round(1)

  total_predicted_winners_ev = ((target_persons.winner_ev * target_persons.weight *
                              target_persons.pred_vote).sum())
  
  percent_predicted_voters_ev = ((total_predicted_winners_ev / target_predicted_population) * 100).round(1)

  return pd.Series([percent_total_winners, 
                    percent_eligible_winners, 
                    percent_predicted_voters,
                    percent_total_winners_ev, 
                    percent_eligible_winners_ev, 
                    percent_predicted_voters_ev])
  
def winners_state_row(row):
  return winners_state(row.state)

# Create a dataframe with all the US states + US
states = person.state.unique().tolist()
summary = mdf.cartesian_product({'state':['US'] + states})

# Apply the percent winners calculation over the dataframe
summary[['overall', 'eligible_voters','predicted_voters', 
         'overall_ev', 'eligible_voters_ev', 
         'predicted_voters_ev']] = summary.apply(winners_state_row, axis=1)

# Create US dataframe for barchart
us = summary[summary['state'] == 'US']
x = ['All Americans', 'Adult Citizens']
overall = [us.loc[0, 'overall'], us.loc[0, 'overall_ev']]
eligible = [us.loc[0, 'eligible_voters'], us.loc[0, 'eligible_voters_ev']]
predicted = [us.loc[0, 'predicted_voters'], us.loc[0, 'predicted_voters_ev']]

fig = go.Figure(data=[
    go.Bar(name='All Americans', x=x, y=overall, text=overall, marker_color=BLUE),
    go.Bar(name='Eligible Voters', x=x, y=eligible, text=eligible, marker_color=LIGHT_BLUE),
    go.Bar(name='Preicted Voters', x=x, y=predicted, text=predicted, marker_color=GRAY)
])

# Create US grouped bar chart
fig.update_layout(barmode='group',
                  title='Population shares benefiting from UBI policies',
                  xaxis_ticksuffix='',
                  font=dict(family='Roboto'),
                  plot_bgcolor='white',
                 )

fig.update_xaxes(
        tickangle = 0,
        title_text = "UBI recipients",
        tickfont = {"size": 14},
        title_standoff = 25)

fig.update_yaxes(
        title_text = "Percent winners",
        ticksuffix ="%",
        tickfont = {'size':14},
        title_standoff = 25,
        range=[0,70])

fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1
))

fig.update_traces(texttemplate='%{text}%')


format_fig(fig)

Cook Political Report has identified [six key 2022 Senate races](https://cookpolitical.com/ratings/senate-race-ratings): Arizona, Georgia, North Carolina, Ohio, Pennsylvania, and Wisconsin. In each of the six battleground states a majority of predicted voters are net winners from the budget neutral UBI and flat tax. A higher share of predicted voters are better off when children and non-citizens are excluded.

In [ ]:
# Create dataframe for swing states chart
swing = summary[(summary["state"] == 'AZ') | (summary["state"] == 'GA')
 | (summary["state"] == 'NC') | (summary["state"] == 'OH')
  | (summary["state"] == 'PA') | (summary["state"] == 'WI')]

swing = swing.sort_values(by='state',ascending=True)

swing_states = ['Arizona', 'Georgia', 'North Carolina',
                'Ohio', 'Pennsylvania', 'Wisconsin']

overall_ev = [swing.loc[44, 'overall_ev'], swing.loc[29, 'overall_ev'], 
           swing.loc[27, 'overall_ev'], swing.loc[10, 'overall_ev'],
           swing.loc[9, 'overall_ev'], swing.loc[14, 'overall_ev']]
           

eligible_ev = [swing.loc[44, 'eligible_voters_ev'], swing.loc[29, 'eligible_voters_ev'],
            swing.loc[27, 'eligible_voters_ev'], swing.loc[10, 'eligible_voters_ev'],
            swing.loc[9, 'eligible_voters_ev'], swing.loc[14, 'eligible_voters_ev']]

predicted_ev = [swing.loc[44, 'predicted_voters_ev'], swing.loc[29, 'predicted_voters_ev'],
             swing.loc[27, 'predicted_voters_ev'], swing.loc[10, 'predicted_voters_ev'],
             swing.loc[9, 'predicted_voters_ev'], swing.loc[14, 'predicted_voters_ev']]

overall = [swing.loc[44, 'overall'], swing.loc[29, 'overall'], 
           swing.loc[27, 'overall'], swing.loc[10, 'overall'],
           swing.loc[9, 'overall'], swing.loc[14, 'overall']]
           

eligible = [swing.loc[44, 'eligible_voters'], swing.loc[29, 'eligible_voters'],
            swing.loc[27, 'eligible_voters'], swing.loc[10, 'eligible_voters'],
            swing.loc[9, 'eligible_voters'], swing.loc[14, 'eligible_voters']]

predicted = [swing.loc[44, 'predicted_voters'], swing.loc[29, 'predicted_voters'],
             swing.loc[27, 'predicted_voters'], swing.loc[10, 'predicted_voters'],
             swing.loc[9, 'predicted_voters'], swing.loc[14, 'predicted_voters']]

# Create swing states bar chart
fig = go.Figure()

fig.add_trace(go.Bar(name='All Americans', x=swing_states, y=overall_ev, text=overall_ev, marker_color=BLUE))
fig.add_trace(go.Bar(name='Eligible Voters', x=swing_states, y=eligible_ev, text=eligible_ev, marker_color=LIGHT_BLUE))
fig.add_trace(go.Bar(name='Predicted Voters', x=swing_states, y=predicted_ev, text=predicted_ev, marker_color=GRAY))

fig.add_trace(go.Bar(name='All Americans', x=swing_states, y=overall, text=overall, marker_color=BLUE, visible = False))
fig.add_trace(go.Bar(name='Eligble Voters', x=swing_states, y=eligible, text=eligible, marker_color=LIGHT_BLUE, visible = False))
fig.add_trace(go.Bar(name='Predicted Voters', x=swing_states, y=predicted, text=predicted, marker_color=GRAY, visible = False))


fig.update_layout(uniformtext_minsize=10, uniformtext_mode='hide', plot_bgcolor='white')
fig.update_traces(texttemplate='%{text}%', textposition='outside')

fig.update_xaxes(
        tickangle = 0,
        title_text = "State",
        tickfont = {"size": 14},
        title_standoff = 25)

fig.update_yaxes(
        title_text = "Percent winners",
        ticksuffix ="%",
        tickfont = {'size':14},
        title_standoff = 25,
        range=[0,75])

fig.update_xaxes(title_font=dict(size=14, family='Roboto', color='black'))
fig.update_yaxes(title_font=dict(size=14, family='Roboto', color='black'))
fig.update_layout(title_text='Population shares benefiting from UBI policies in six 2022 battleground states')

fig.update_layout(barmode='group')

fig.update_layout(
    updatemenus=[go.layout.Updatemenu(
        active=0,
        buttons=list([
            dict(label="UBI to adult citizens",
                 method="update",
                 args=[{'visible':[True,True,True,False,False,False]},
                       {'title':'Population shares benefiting from UBI policies in six 2022 battleground states',
                        'showlegend':True}]),
            
            dict(label="UBI to all Americans",
                 method="update",
                 args=[{'visible':[False, False,False,True, True, True]},
                       {'title':'Population shares benefiting from UBI policies in six 2022 battleground states',
                        'showlegend':True}]),
                       ]),
        
            direction="down",
            pad={"r": 10, "t": 10},
            showactive=True,
            x=-0.35,
            xanchor="left",
            y=1.1,
            yanchor="top"
    
    )])

fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1
))


format_fig(fig)

In total, a flat tax funded UBI benefits the majority of likely voters in 36 states as shown in the choropleth map below.

In [ ]:
# Create cutoff point for choropleth chart
summary['majority'] = summary.predicted_voters > 50

# Create choropleth map
fig = px.choropleth(summary, 
              locations = 'state',
              color="predicted_voters", 
              color_continuous_scale="PRGn",
              color_continuous_midpoint=50,
              locationmode='USA-states',
              scope="usa",
              title='Share of predicted voters who benefit from flat tax funded UBI',
              height=600,
              labels={'predicted_voters': "Percent Winners",
                      'state':'State',
                    }
             )
fig.update_layout(coloraxis_showscale=False)

format_fig(fig)

# Conclusion

In my model, a flat income tax funded UBI benefits 65 percent of voters. In a previous [analysis](https://www.ubicenter.org/budget-neutral-version-of-andrew-yangs-freedom-dividend), Max Ghenis found similar results with a budget-neutral version of Andrew Yang’s Freedom Dividend (68 percent of people were winners). However, as a tool to win elections, campaigns can increase the share of eligible and predicted voters that are better off by excluding non-voting populations such as children and non-citizens. These results highlight the tensions between optimal policy and optimal politics and leave many open questions for campaigns and politicians to answer.

# Appendix

**Figure 5: Population shares benefiting from UBI policies in all states and Washington, D.C.**

In [ ]:
summary = summary.drop(['majority'], axis=1)
summary = summary.rename(columns={"state": "State"})
summary = summary.set_index('State')
summary.columns = pd.MultiIndex.from_product([['UBI including all Americans', 
                                               'UBI excluding children and non-citizens'],
                                               ['Overall', 'Eligible Voters', 
                                                'Predicted Voters']],
                                     names=['', ''])
summary